In [1]:
from keras.datasets import mnist
from keras.layers import Input,Dense, Conv2D,MaxPooling2D,Dropout,Flatten
from keras.models import Model
import keras.backend as K
from keras.utils import np_utils
from keras.callbacks import TensorBoard
import cv2
import numpy as np

C:\Users\ie-02\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt

In [3]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1,
                          write_graph=True, write_images=False)

In [4]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0],28, 28,1).astype('float32') 
X_test = X_test.reshape(X_test.shape[0],28, 28,1).astype('float32')


X_train = X_train / 255
X_test = X_test / 255

In [5]:
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)
num_classes = Y_test.shape[1]


In [6]:
input_layer = Input(shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]))
conv_1 = Conv2D(64,(3,3),activation = 'relu')(input_layer)
pooling_1 = MaxPooling2D(pool_size=(2,2))(conv_1)
dropout_1 = Dropout(0.25)(pooling_1)
conv_2 = Conv2D(64,(3,3),activation = 'relu')(dropout_1)
dropout_2 = Dropout(0.25)(conv_2)
flat_1 = Flatten()(dropout_2)
fcl_1 = Dense(64,activation = 'relu')(flat_1)
dropout_3 = Dropout(0.25)(fcl_1)
fcl_2 = Dense(64,activation = 'relu')(dropout_3)
dropout_4 = Dropout(0.25)(fcl_2)
res = Dense(10,activation = 'softmax')(dropout_4)
model = Model(inputs=input_layer,outputs = res)

In [7]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])

In [8]:
model.load_weights("mnist_model1.h5")

In [9]:
model